In [ ]:
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import time
import pymysql

import pandas as pd
from sqlalchemy import create_engine
from PIL import Image
import base64
from io import BytesIO

from glob import glob
import subprocess
import sys

googleID = 'applemangoHansung@gmail.com'
googlePW = 'applemango!@#123'

smtp = smtplib.SMTP('smtp.gmail.com', 587) 
smtp.ehlo()      
smtp.starttls()  
smtp.login(googleID, googlePW)

def sendEmail(title, text, sendID = 'cjfwoqkr0105@gmail.com'):
    msg = MIMEText(text)
    msg['Subject'] = title
    msg['To'] = sendID
    smtp.sendmail(googleID, sendID, msg.as_string())

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

preTime = datetime.now()
try:
    while True:
        nowTime = datetime.now()
        print(preTime, nowTime)
            
        # 냉장고 내의 음식의 유통기한이 임박, 초과하면 주인들에게 메일
        # 카메라 마다 최신 사진을 찾은 뒤, 식품의 유통기한과 넣은 시간을 현재 시간과 비교 
        sql = "select * from camera"
        curs.execute(sql)
        rows = curs.fetchall()
        print(rows)
        
        for row in rows:      
            sql = "select photoID from photo where cameraID = '" + row[0] + "'"
            curs.execute(sql)
            photos = curs.fetchall()
            print(photos)
            
            late = ' '
            if len(photos) != 0:
                late = photos[0][0];       
                late_split = late.split('_') 
                late_time = datetime(int(late_split[0]), int(late_split[1]), int(late_split[2]), int(late_split[3]), int(late_split[4]), int(late_split[5]))
                for photo in photos :
                    photo_split = photo[0].split('_')
                    photo_time = datetime(int(photo_split[0]), int(photo_split[1]), int(photo_split[2]), int(photo_split[3]), int(photo_split[4]), int(photo_split[5]))
                    if late_time < photo_time :
                        late = photo[0]
                        late_split = late.split('_')
                        late_time = datetime(int(late_split[0]), int(late_split[1]), int(late_split[2]), int(late_split[3]), int(late_split[4]), int(late_split[5]))
            print(late)
            
            sql = "select * from refrigerator where raspberryPiID = '" + row[1] + "'"
            curs.execute(sql)
            refrigerator = curs.fetchall()
            print(refrigerator)
            
            sql = "select * from possession where raspberryPiID = '" + refrigerator[0][0] + "'"
            curs.execute(sql)
            possessions = curs.fetchall()
            print(possessions)
            
            for possession in possessions:
                email = possession[0]
                
                sql = "select * from members where email = '" + email + "'"
                curs.execute(sql)
                member = curs.fetchall()
                print(member)
                
                if late != ' ':
                    sql = "select * from photographedFood where photoID = '" + late + "'"
                    curs.execute(sql)
                    foods = curs.fetchall()
                    print(foods)
                    
                    title = ''
                    text = ''
                    for food in foods:
                        foodName = food[2]
                        shelfLife = food[3]
                        inday_split = food[4].split('_')
                        inTime = datetime(int(inday_split[0]), int(inday_split[1]), int(inday_split[2]), int(inday_split[3]), int(inday_split[4]), int(inday_split[5]))
                        diff = (nowTime-inTime).days
                        # 유통기한 초과 시 메일
                        if diff > shelfLife: # 보낸 기록 확인 추가
                            title = '유통기한이 초과 및'
                            text = text + refrigerator[0][2] + '냉장고의 ' + foodName + '식품의 유통기한 초과! \n'
                        # 유통기한 임박 시 메일
                        elif diff >= shelfLife - 3 or shelfLife < 3 : # 보낸 기록 확인 추가
                            title = '유통기한이 임박 및'
                            text = text + refrigerator[0][2] + '냉장고의 ' + foodName + '식품의 유통기한 임박! \n'    
                        
                        if title != '':
                            # yolo실행
                            # 레시피를 받아옴
                            # text에 레시피 추천 추가
                            text = text + '오늘의 추천 레시피입니다.\n'
                            for recipe in recipes:
                                text = text + recipe[0] + ': http://project/recipe/' + recipe[0] + '\n'
                            title = title + ' 레시피 추천 메일입니다.'
                            sendEmail(title, text, email)
                
        # images 하루 한 번 전체 삭제
        if nowTime.hour == 0 and preTime.hour == 23:
            sql = "delete from images"
            curs.execute(sql)
            conn.commit()
            
        # eatenFood 24시간이 지난 음식 삭제
        sql = "select * from eatenFood"
        curs.execute(sql)
        rows = curs.fetchall()

        for row in rows:
            photoID = row[0]
            row_split = photoID.split('_')
            row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
            if (nowTime - row_time).days > 0:
                sql = "delete from eatenFood where photoID = '" + photoID + "'"
                curs.execute(sql)
                conn.commit()
        
        # photo 테이블에 저장한지 30일 초과한 data와 analyzedData 삭제
        sql = "select * from photo"
        curs.execute(sql)
        rows = curs.fetchall()
        
        for row in rows:
            photoID = row[0]
            row_split = photoID.split('_')
            row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
            if (nowTime - row_time).days > 30:
                sql = "update photo set data = 'NULL', analyzedData = 'NULL' where = '" + photoID + "'"
                curs.execute(sql)
                conn.commit()
                
        preTime = nowTime
        
        # 1분에 한번 검사
        time.sleep(60)
except KeyboardInterrupt:
       pass
smtp.quit()
conn.close()

In [ ]:
#설정한 시간에 레시피 추천 메일

import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import time
import pymysql

googleID = 'applemangoHansung@gmail.com'
googlePW = 'applemango!@#123'

smtp = smtplib.SMTP('smtp.gmail.com', 587) 
smtp.ehlo()      
smtp.starttls()  
smtp.login(googleID, googlePW)

def sendEmail(title, text, sendID = 'cjfwoqkr0105@naver.com'):
    msg = MIMEText(text)
    msg['Subject'] = title
    msg['To'] = sendID
    smtp.sendmail(googleID, sendID, msg.as_string())

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

nowTime = datetime.now()
print(nowTime)

sql = "select * from members"
curs.execute(sql)
members = curs.fetchall()

for member in members:
    email = member[3]
    sql = "select recipeName from recommendRecipe where email = '" + email + "'"
    curs.execute(sql)
    recipes = curs.fetchall()

    title = '오늘의 레시피 추천'
    text = '오늘의 추천 레시피입니다.\n'
    for recipe in recipes:
        text = text + recipe[0] + ': http://project/recipe/' + recipe[0] + '\n'
    sendEmail(title, text, email)

smtp.quit()
conn.close()

In [ ]:
import pymysql
from datetime import datetime, timedelta

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

sql = "select photoID from photo where cameraID=%s"
curs.execute(sql, 'camera1')
rows = curs.fetchall()
print(rows)

now = '2020_4_21_14_6_32'
now_split = now.split('_')
now_time = datetime(int(now_split[0]), int(now_split[1]), int(now_split[2]), int(now_split[3]), int(now_split[4]), int(now_split[5]))
    
pre = ''
if len(rows) != 0:
    pre = rows[0][0];       
    pre_split = pre.split('_') 
    pre_time = datetime(int(pre_split[0]), int(pre_split[1]), int(pre_split[2]), int(pre_split[3]), int(pre_split[4]), int(pre_split[5]))
    for row in rows :    
        row_split = row[0].split('_')
        row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
        print(pre_time, row_time)
        if now_time == row_time:
            pass
        elif pre_time < row_time :
            pre = row[0]
            pre_split = pre.split('_')
            pre_time = datetime(int(pre_split[0]), int(pre_split[1]), int(pre_split[2]), int(pre_split[3]), int(pre_split[4]), int(pre_split[5]))
print(pre)
conn.close()

In [ ]:
from datetime import datetime, timedelta
import time


time1_text = '2020_4_13_4_50'
time_split = time1_text.split('_')
time1 = datetime(int(time_split[0]), int(time_split[1]), int(time_split[2]), int(time_split[3]), int(time_split[4]))

time2_text = '2020_4_13_4_50_20'
time_split = time2_text.split('_')
time2 = datetime(int(time_split[0]), int(time_split[1]), int(time_split[2]), int(time_split[3]), int(time_split[4]), int(time_split[5]))
print(time1, time2, time1 == time2)

nowTime = datetime.now()
n = datetime(nowTime.year, nowTime.month, nowTime.day, nowTime.hour, nowTime.minute)
print(time1, n, time1 == n)

2020-06-03 03:14:08.520882 2020-06-03 03:14:08.520882
2020-06-03 03:14:08.520882 2020-06-03 03:14:08.960920
2020-06-03 03:14:08.960920 2020-06-03 03:14:09.093925
2020-06-03 03:14:09.093925 2020-06-03 03:14:09.195934
2020-06-03 03:14:09.195934 2020-06-03 03:14:09.273939
2020-06-03 03:14:09.273939 2020-06-03 03:14:09.348945
2020-06-03 03:14:09.348945 2020-06-03 03:14:09.432941
2020-06-03 03:14:09.432941 2020-06-03 03:14:09.579963
2020-06-03 03:14:09.579963 2020-06-03 03:14:09.651960
2020-06-03 03:14:09.651960 2020-06-03 03:14:09.719974
2020-06-03 03:14:09.719974 2020-06-03 03:14:09.802969
2020-06-03 03:14:09.802969 2020-06-03 03:14:09.923990
2020-06-03 03:14:09.923990 2020-06-03 03:14:10.073342
2020-06-03 03:14:10.073342 2020-06-03 03:14:10.165999
2020-06-03 03:14:10.165999 2020-06-03 03:14:10.255016
2020-06-03 03:14:10.255016 2020-06-03 03:14:10.338011
2020-06-03 03:14:10.338011 2020-06-03 03:14:10.417019
2020-06-03 03:14:10.417019 2020-06-03 03:14:10.504024
2020-06-03 03:14:10.504024 2